In [1]:
# !pip install yfinance

In [28]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from autogluon.tabular import TabularPredictor
from datetime import datetime, timedelta
import numpy as np
import gradio as gr

# 1. 분석할 주식 리스트 (Apple, Microsoft, Amazon, Tesla, Nvidia, AMD)
tickers = {'AAPL': 'Apple', 'MSFT': 'Microsoft', 'AMZN': 'Amazon', 'TSLA': 'Tesla', 'NVDA': 'Nvidia', 'AMD': 'AMD'}

# 2. 날짜 설정 (2014년 1월 1일부터 전날까지)
end_date = datetime.now() - timedelta(days=1)  # 전날까지의 데이터
start_date = datetime.strptime("2014-01-01", "%Y-%m-%d")  # 2014년 1월 1일부터

# 3. 주식 데이터를 불러오는 함수
def load_stock_data(selected_ticker):
    if selected_ticker not in tickers:
        return f"잘못된 티커를 입력하셨습니다."
    
    # yfinance로 주식 데이터 불러오기
    df = yf.download(selected_ticker, start=start_date, end=end_date)
    
    if df.empty:
        return f"{tickers[selected_ticker]}에 대한 데이터가 없습니다."
    
    return df.head(), df.tail()  # 데이터를 head와 tail로 반환

# 4. 분석 및 시각화하는 함수
def analyze_stock(selected_ticker):
    df = yf.download(selected_ticker, start=start_date, end=end_date)

    # 데이터 전처리
    df.reset_index(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day'] = (df['Date'] - df['Date'].min()).dt.days  # 날짜를 숫자로 변환

    # 50일, 200일 이동평균선 계산
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()

    # 매수 및 매도 시점 계산
    df['Signal'] = 0
    df.loc[50:, 'Signal'] = np.where(df['MA50'][50:] > df['MA200'][50:], 1, 0)
    df['Position'] = df['Signal'].diff()

    # AutoGluon을 이용한 종가 예측
    train_data = df[['Day', 'Close']].copy()
    train_data = train_data.rename(columns={'Close': 'label'})
    predictor = TabularPredictor(label='label').fit(train_data)

    best_model = predictor.model_best
    models = predictor.get_model_names()
    all_predictions = [predictor.predict(train_data.drop(columns=['label']), model=model) for model in models]
    mean_predictions = np.mean(np.array(all_predictions), axis=0)
    std_predictions = np.std(np.array(all_predictions), axis=0)

    mape = np.mean(np.abs((train_data['label'] - mean_predictions) / train_data['label'])) * 100

    # 향후 30일 예측값 계산
    future_dates = [end_date + timedelta(days=i) for i in range(1, 31)]
    future_days = [(date - df['Date'].min()).days for date in future_dates]
    future_df = pd.DataFrame({'Day': future_days})
    future_all_predictions = [predictor.predict(future_df, model=model) for model in models]
    future_mean_predictions = np.mean(np.array(future_all_predictions), axis=0)
    future_std_predictions = np.std(np.array(future_all_predictions), axis=0)

    # 시각화 함수
    def plot_graph(data_df, future_dates, future_mean_predictions, future_std_predictions):
        plt.figure(figsize=(14, 8))
        plt.plot(data_df['Date'], data_df['Close'], label=f'{selected_ticker} Actual Close Prices', color='#A1C6EA')  # 파스텔 블루
        plt.plot(data_df['Date'], data_df['MA50'], label='50-Day Moving Average', color='#F4B3C2', linestyle='--')  # 파스텔 핑크
        plt.plot(data_df['Date'], data_df['MA200'], label='200-Day Moving Average', color='#B3D4A7', linestyle='--')  # 파스텔 그린
        plt.plot(data_df[data_df['Position'] == 1]['Date'], data_df[data_df['Position'] == 1]['Close'], '^', markersize=10, color='red', lw=0, label='Buy Signal')
        plt.plot(data_df[data_df['Position'] == -1]['Date'], data_df[data_df['Position'] == -1]['Close'], 'v', markersize=10, color='blue', lw=0, label='Sell Signal')
        plt.plot(future_dates, future_mean_predictions, label='Future Predicted Prices', color='#B3D4A7', linestyle='--')
        plt.fill_between(future_dates, future_mean_predictions - future_std_predictions, future_mean_predictions + future_std_predictions, color='#B3D4A7', alpha=0.2)
        plt.xlabel('Date')
        plt.ylabel('Stock Price')
        plt.legend()
        plt.tight_layout()
        
        # Gradio에서 그래프를 반환할 수 있도록 설정
        return plt.gcf()

    # 그래프 반환
    return plot_graph(df, future_dates, future_mean_predictions, future_std_predictions), f"{tickers[selected_ticker]} 분석 완료, MAPE: {mape:.2f}%"


# Gradio 인터페이스 정의

# 주가 보기 기능 인터페이스
def stock_view(selected_ticker):
    df_head, df_tail = load_stock_data(selected_ticker)
    return df_head, df_tail

# 주가 분석 기능 인터페이스
def stock_analysis(selected_ticker):
    return analyze_stock(selected_ticker)

# Gradio UI 설정
app = gr.Blocks()

with app:
    gr.Markdown("## 주식 데이터 조회 및 분석")
    
    stock_ticker_dropdown = gr.Dropdown(choices=list(tickers.keys()), label="주식을 선택하세요", value="AAPL")
    
    with gr.Row():
        stock_ticker_dropdown
    
    df_head_output = gr.Dataframe(label="Head 데이터")
    df_tail_output = gr.Dataframe(label="Tail 데이터")
    
    stock_view_button = gr.Button("주가 보기")
    
    with gr.Row():
        stock_view_button
    
    with gr.Row():
        df_head_output
        df_tail_output
    
    graph_output = gr.Plot(label="주가 분석 결과")
    analysis_text_output = gr.Textbox(label="분석 결과")
    
    stock_analysis_button = gr.Button("주식 분석하기")
    
    with gr.Row():
        stock_analysis_button
    
    with gr.Row():
        graph_output
        analysis_text_output

    # Gradio 이벤트 정의
    stock_view_button.click(view_stock_data, inputs=stock_ticker_dropdown, outputs=[df_head_output, df_tail_output])
    stock_analysis_button.click(analyze_stock_data, inputs=stock_ticker_dropdown, outputs=[graph_output, analysis_text_output])

# Gradio 앱 실행
app.launch(inbrowser=True)


INFO:httpx:HTTP Request: GET http://127.0.0.1:7863/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7863/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
No path specified. Models will be saved in: "AutogluonModels/ag-20241017_085143"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          8
Memory Avail:       12.06 GB / 14.61 GB (82.5%)
Disk Space Avail:   20.38 GB / 223.03 GB (9.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_qualit

[1000]	valid_set's rmse: 6.05719
[2000]	valid_set's rmse: 5.0402
